In [1]:
!git clone "https://github.com/Kasper-Raupach-Haurum/M6-Data-Engineering-and-MLOps.git"

Cloning into 'M6-Data-Engineering-and-MLOps'...


In [2]:
%pip install mlflow --quiet
!mlflow
import os
import shutil
import pprint
import sys
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np

from random import random, randint
import mlflow
import mlflow.sklearn
mlflow.set_experiment('HR')

from mlflow import log_metric, log_param, log_artifacts
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from mlflow.tracking import MlflowClient
import warnings

warnings.filterwarnings("ignore")
print(mlflow.__version__)

Note: you may need to restart the kernel to use updated packages.
Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect recipe results.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


2023/04/29 10:55:24 INFO mlflow.tracking.fluent: Experiment with name 'HR' does not exist. Creating a new experiment.


2.3.0


In [3]:
path = 'M6-Data-Engineering-and-MLOps\general_data.csv'
HR = pd.read_csv(path)
path1 = 'M6-Data-Engineering-and-MLOps\employee_survey_data.csv'
ES = pd.read_csv(path1)
path2 = 'M6-Data-Engineering-and-MLOps\manager_survey_data.csv'
MS = pd.read_csv(path2)
EMS = pd.merge(ES, MS)
hr_df = pd.merge(HR, EMS)

In [4]:
hrdf = hr_df.drop_duplicates(subset = ['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'Gender', 'JobLevel', 'JobRole', 'MaritalStatus', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager'])

hrdf.NumCompaniesWorked = hrdf.NumCompaniesWorked.fillna(3)
hrdf.EnvironmentSatisfaction = hrdf.EnvironmentSatisfaction.fillna(3)
hrdf.JobSatisfaction = hrdf.JobSatisfaction.fillna(3)
hrdf.WorkLifeBalance = hrdf.WorkLifeBalance.fillna(3)
hrdf.TotalWorkingYears = hrdf.TotalWorkingYears.fillna(11)

hrdf['NumCompaniesWorked'] = hrdf['NumCompaniesWorked'].astype(int)
hrdf['TotalWorkingYears'] = hrdf['TotalWorkingYears'].astype(int)
hrdf['EnvironmentSatisfaction'] = hrdf['EnvironmentSatisfaction'].astype(int)
hrdf['JobSatisfaction'] = hrdf['JobSatisfaction'].astype(int)
hrdf['WorkLifeBalance'] = hrdf['WorkLifeBalance'].astype(int)

hrdf['Attrition'].replace({'No' : '0'}, inplace=True)
hrdf['Attrition'].replace({'Yes' : '1'}, inplace=True)

hrdf['NumCompaniesWorked'] = hrdf['NumCompaniesWorked'].astype(int)
hrdf['TotalWorkingYears'] = hrdf['TotalWorkingYears'].astype(int)
hrdf['EnvironmentSatisfaction'] = hrdf['EnvironmentSatisfaction'].astype(int)
hrdf['JobSatisfaction'] = hrdf['JobSatisfaction'].astype(int)
hrdf['WorkLifeBalance'] = hrdf['WorkLifeBalance'].astype(int)
hrdf['Attrition'] = hrdf['Attrition'].astype(int)

hrdf.drop(['BusinessTravel', 'EmployeeCount', 'Department',
            'StandardHours', 'Over18'], axis=1, inplace=True)
print('Size of Full dataset is: {}'.format(hrdf.shape))

hrdf.drop(['EducationField', 'Gender',
            'JobRole', 'MaritalStatus'], axis=1, inplace=True)
print('Size of Full dataset is: {}'.format(hrdf.shape))

Size of Full dataset is: (1498, 24)
Size of Full dataset is: (1498, 20)


In [5]:
def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

In [6]:
def train_model(alpha, l1_ratio):
    train, test = train_test_split(hrdf)
    train_x = train.drop(["Attrition"], axis=1)
    test_x = test.drop(["Attrition"], axis=1)
    train_y = train[["Attrition"]]
    test_y = test[["Attrition"]]

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

In [7]:
train_model(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.36593089238244136
  MAE: 0.2688843565077428
  R2: 0.029658383684258993


In [8]:
train_model(0.5, 0.4)

Elasticnet model (alpha=0.500000, l1_ratio=0.400000):
  RMSE: 0.3548212331699813
  MAE: 0.26235830933305565
  R2: 0.008932973110728937


In [9]:
train_model(0.5, 0.3)

Elasticnet model (alpha=0.500000, l1_ratio=0.300000):
  RMSE: 0.36318937886620417
  MAE: 0.26632988025071985
  R2: 0.03156755418991053


In [10]:
train_model(0.5, 0.2)

Elasticnet model (alpha=0.500000, l1_ratio=0.200000):
  RMSE: 0.36605258366627774
  MAE: 0.2678123404640618
  R2: 0.05330708802343809


In [11]:
train_model(0.5, 0.1)

Elasticnet model (alpha=0.500000, l1_ratio=0.100000):
  RMSE: 0.3368023893291754
  MAE: 0.25140833320072437
  R2: 0.06528089007330662
